In [ ]:
#Only run in colab
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Github/Taskmaster
#!pip install tensorflow --upgrade
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

In [1]:
import small_rubiks as rubiks
import numpy as np
from student import student
from classroom import classroom
from teacher import teacher
from small_rubiks_neural_networks import student_network

task = rubiks.rubiks_task
setup = rubiks.rubiks_setup
rng = np.random.default_rng(seed=0)

#my_student_network = student_network.load("models/trained.h5")
my_student_network = student_network.create(residual_layers = 4)
student_template = student(task, 50,my_student_network)
t = teacher(setup, lambda n : 1+rng.poisson(lam=3,size=n))
c = classroom(task, setup, t, student_template, n_students=1, max_steps=10)
#c.run_training_batch(n_problems=1,epochs_per_episode=3)


In [2]:
for i in range(10):
    c.run_training_batch(n_problems=1,epochs_per_episode=3)
    #if i % 10 == 0:
    #    my_student_network.save("models/t.keras")

Finished step 0, 1 out of 1 remain open.
Finished step 1, 1 out of 1 remain open.
Finished step 2, 1 out of 1 remain open.
Finished step 3, 1 out of 1 remain open.
Finished step 4, 1 out of 1 remain open.
Finished step 5, 1 out of 1 remain open.
Finished step 6, 1 out of 1 remain open.
Finished step 7, 1 out of 1 remain open.
Finished step 8, 1 out of 1 remain open.
Finished step 9, 1 out of 1 remain open.


AttributeError: 'student_network' object has no attribute 'fit'

In [1]:
from small_rubiks_neural_networks import student_network
import numpy as np
import small_rubiks as rubiks
from utils import dotdict
from keras import regularizers

params = dotdict({
    "residual_weights_reg" : regularizers.l2(l2=0.001),
    "residual_bias_reg" : regularizers.l2(0.001),
    "relu_leak" : 0.1,
    "residual_units" : 2000,
    "learning_rate" : 0.001,
    "residual_layers" : 0
})

my_student_network = student_network()
my_student_network.build_state_network(params)

n=4096
rng = np.random.default_rng(seed=0)
states = np.arange(24)[None,:].repeat(n,axis=0)

for k in range(15):
    for i in range(30):
        _, states = rubiks.task_action(states,rng.choice(a=12,size=n))

    actions = rng.choice(12,size=n)
    _, next_states = rubiks.task_action(states,actions)
    my_student_network.fit_state(states,actions,next_states,epochs = 10)



Epoch 1/10

120/120 [==============================] - 4s 10ms/step - loss: 2.6523 - val_loss: 1.9639
Epoch 2/10
120/120 [==============================] - 1s 7ms/step - loss: 1.5063 - val_loss: 1.2337
Epoch 3/10
120/120 [==============================] - 1s 7ms/step - loss: 0.9695 - val_loss: 0.8585
Epoch 4/10
120/120 [==============================] - 1s 8ms/step - loss: 0.7035 - val_loss: 0.6694
Epoch 5/10
120/120 [==============================] - 1s 9ms/step - loss: 0.5581 - val_loss: 0.5641
Epoch 6/10
120/120 [==============================] - 1s 9ms/step - loss: 0.4779 - val_loss: 0.5015
Epoch 7/10
120/120 [==============================] - 1s 10ms/step - loss: 0.4370 - val_loss: 0.4718
Epoch 8/10
120/120 [==============================] - 1s 9ms/step - loss: 0.4127 - val_loss: 0.4546
Epoch 9/10
120/120 [==============================] - 1s 9ms/step - loss: 0.3980 - val_loss: 0.4511
Epoch 10/10
120/120 [==============================] - 1s 8ms/step - loss: 0.3859 - val_loss: 0

In [8]:
rubiks.print_coloring(states[0])
for i in range(12):
    rubiks.print_coloring(rubiks.task_action(states[0],i))

  BY    
  OO    
RBWWGBOY
GRGBWWGY
  YR    
  OR    

  WY    
  GO    
BRYWGBOO
RGOBWWGB
  YR    
  YR    

  YY    
  YO    
GRBWGBOO
RBOBWWGY
  WR    
  GR    

  BW    
  OB    
RBWRWGOY
GRGRWBYY
  YG    
  OO    

  BG    
  OO    
RBWYBWRY
GRGOGWRY
  YW    
  OB    

  BY    
  OO    
RBWWGBOY
GYGRGBWW
  OY    
  RR    

  BY    
  OO    
RBWWGBOY
GBWWGYGR
  RR    
  YO    

  YO    
  BO    
OYRBWWGB
GRGBWWGY
  YR    
  OR    

  OB    
  OY    
WWGBOYRB
GRGBWWGY
  YR    
  OR    

  BY    
  GW    
ROWBRBOY
GOWGYWGY
  BR    
  OR    

  BY    
  RB    
RYGWOBOY
GRBWOWGY
  WG    
  OR    

  BW    
  OO    
YBWWGRGO
BRGBWOYY
  YR    
  RG    

  GR    
  OO    
OBWWGBYY
RRGBWYOG
  YR    
  WB    



In [5]:
states = np.arange(24)[None,:]
actions = np.array([0])
prediction = my_student_network.predict_state(states,actions)
prediction.reshape(1,24,6).argmax(axis=2), rubiks.start_coloring[rubiks.task_action(states,actions)[1][0]]

1/1 [==============================] - 0s 26ms/step


(array([[0, 1, 2, 0, 0, 5, 0, 5, 3, 0, 4, 3, 1, 2, 4, 1, 2, 5, 1, 5, 2, 1,
         4, 3]], dtype=int64),
 array([0, 5, 2, 0, 5, 3, 0, 4, 2, 0, 4, 3, 1, 2, 4, 1, 2, 5, 1, 3, 4, 1,
        3, 5]))